# 새로나온책 (1~10페이지 총 200권)
-제목, 저자, 번역자, 출간일, 페이지, 정가, 판매가, 마일리지를 크롤링

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
from urllib.parse import quote

In [181]:
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html?page='
site = 'https://www.hanbit.co.kr/'
req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

In [182]:
lis =soup.select(".sub_book_list_area > li")  
len(lis)

20

In [183]:
li = lis[2]
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B3128413784'

In [184]:
sub_url = site + href
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [185]:
# 제목
name = li.select_one('.book_tit').string
name

'당당한 디자인 결정을 위한 9가지 방법'

In [186]:
# 저자
artist = li.select_one('.book_writer').string
artist

'톰 그리버 '

In [187]:
# 번역
if len(sub_soup.select_one('.info_list').find_all('li')) == 6:
    translate = sub_soup.select_one('.info_list').find_all('li')[1].find('span').string
else:
    translate = '없음'
translate

'김민성 , 곽서희 '

In [188]:
#출간일
day = sub_soup.select_one('.info_list').find_all('li')[-4].find('span').string
day

'2021-12-10'

In [189]:
#페이지
page = sub_soup.select_one('.info_list').find_all('li')[-3].find('span').string
page

'376 쪽'

In [190]:
# 정가
price = sub_soup.select_one('.pbr').find('del').string
price

'22,000원'

In [191]:
# 판매가
cost = sub_soup.select('.pbr')[1].find('strong').string
cost

'19,800'

In [192]:
# 마일리지
mile = sub_soup.select('.pbr')[-1].string
mile

'1,100점 (5%)'

#### 1페이지 확인 겸 출력

In [205]:
lines = []
for li in lis:
    href = li.find('a')['href']
    name = li.select_one('.book_tit').string
    artist = li.select_one('.book_writer').string

    sub_url = site + href
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')

    if len(sub_soup.select_one('.info_list').find_all('li')) == 6:
        translate = sub_soup.select_one('.info_list').find_all('li')[1].find('span').string
    else:
        translate = '없음'
    day = sub_soup.select_one('.info_list').find_all('li')[-4].find('span').string
    page = sub_soup.select_one('.info_list').find_all('li')[-3].find('span').string
    price = sub_soup.select_one('.pbr').find('del').string
    cost = sub_soup.select('.pbr')[1].find('strong').string
    mile = sub_soup.select('.pbr')[-1].string

    lines.append([name, artist, translate, day, page, price, cost, mile])
    

In [194]:
df = pd.DataFrame(lines, columns =['책제목', '저자', '역자', '출간일', '쪽수','정가','판매가', '마일리지'])
df.head()

,책제목,저자,역자,출간일,쪽수,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",2021-12-14,416 쪽,"19,800원","17,820",990점 (5%)
1,"다시, 배우다",폴 김,없음,2021-12-10,304 쪽,"16,000원","14,400",800점 (5%)
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",2021-12-10,376 쪽,"22,000원","19,800","1,100점 (5%)"
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",없음,2021-12-01,384 쪽,"18,000원","16,200",900점 (5%)
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",2021-11-30,600 쪽,"36,000원","36,000","1,080점 (3%)"


In [195]:
df.tail()

,책제목,저자,역자,출간일,쪽수,정가,판매가,마일리지
15,나도 영문 손글씨 잘 쓰면 소원이 없겠네,윤정희(리제 캘리그라피),없음,2021-11-05,240 쪽,"16,800원","15,120",840점 (5%)
16,구글 BERT의 정석,수다르산 라비찬디란,"전희원 , 정승환 , 김형준",2021-11-03,368 쪽,"34,000원","30,600","1,700점 (5%)"
17,1000개 숨은그림찾기 몬스터 : 찾아도 찾아도 끝판왕,김확실,없음,2021-11-02,96 쪽,"8,000원","7,200",400점 (5%)
18,1000개 숨은그림찾기 보물찾기 : 찾아도 찾아도 끝판왕,전진희,없음,2021-11-02,96 쪽,"8,000원","7,200",400점 (5%)
19,1000개 숨은그림찾기 탈것 : 찾아도 찾아도 끝판왕,이한이,없음,2021-11-02,96 쪽,"8,000원","7,200",400점 (5%)


### 1에서 10페이지까지 출력하기

In [206]:
lines2 = []
for page in range(1,11):
    'https://www.hanbit.co.kr/store/books/new_book_list.html?page='+str(page)
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    href = li.find('a')['href']
    for li in lis:
        href = li.find('a')['href']
        name = li.select_one('.book_tit').string
        artist = li.select_one('.book_writer').string

        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')

        if len(sub_soup.select_one('.info_list').find_all('li')) == 6:
            translate = sub_soup.select_one('.info_list').find_all('li')[1].find('span').string
        else:
            translate = '없음'
        day = sub_soup.select_one('.info_list').find_all('li')[-4].find('span').string
        page = sub_soup.select_one('.info_list').find_all('li')[-3].find('span').string
        price = sub_soup.select_one('.pbr').find('del').string
        cost = sub_soup.select('.pbr')[1].find('strong').string
        mile = sub_soup.select('.pbr')[-1].string

        lines2.append([name, artist, translate, day, page, price, cost, mile])

In [257]:
df2 = pd.DataFrame(lines2, columns =['책제목', '저자', '역자', '출간일', '쪽수','정가','판매가', '마일리지'])
df2.head()

,책제목,저자,역자,출간일,쪽수,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",2021-12-14,416 쪽,"19,800원","17,820",990점 (5%)
1,"다시, 배우다",폴 김,없음,2021-12-10,304 쪽,"16,000원","14,400",800점 (5%)
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",2021-12-10,376 쪽,"22,000원","19,800","1,100점 (5%)"
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",없음,2021-12-01,384 쪽,"18,000원","16,200",900점 (5%)
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",2021-11-30,600 쪽,"36,000원","36,000","1,080점 (3%)"


In [258]:
df2.tail()

,책제목,저자,역자,출간일,쪽수,정가,판매가,마일리지
195,나도 영문 손글씨 잘 쓰면 소원이 없겠네,윤정희(리제 캘리그라피),없음,2021-11-05,240 쪽,"16,800원","15,120",840점 (5%)
196,구글 BERT의 정석,수다르산 라비찬디란,"전희원 , 정승환 , 김형준",2021-11-03,368 쪽,"34,000원","30,600","1,700점 (5%)"
197,1000개 숨은그림찾기 몬스터 : 찾아도 찾아도 끝판왕,김확실,없음,2021-11-02,96 쪽,"8,000원","7,200",400점 (5%)
198,1000개 숨은그림찾기 보물찾기 : 찾아도 찾아도 끝판왕,전진희,없음,2021-11-02,96 쪽,"8,000원","7,200",400점 (5%)
199,1000개 숨은그림찾기 탈것 : 찾아도 찾아도 끝판왕,이한이,없음,2021-11-02,96 쪽,"8,000원","7,200",400점 (5%)


### 쪽수와 정가, 판매가, 마일리지 정수로 바꾸기
- 오래걸리는 관계로 1페이지만

In [249]:
def convert_units(s):
    return int(s.replace('쪽', '').replace('원','').replace(',',''))

In [218]:
convert_units(page)

96

In [252]:
mile = sub_soup.select('.pbr')[-1].string
int(mile.split('점')[0].replace(',',''))

1100

In [253]:
lines3 = []
for li in lis:
    href = li.find('a')['href']
    name = li.select_one('.book_tit').string
    artist = li.select_one('.book_writer').string

    sub_url = site + href
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')

    if len(sub_soup.select_one('.info_list').find_all('li')) == 6:
        translate = sub_soup.select_one('.info_list').find_all('li')[1].find('span').string
    else:
        translate = '없음'
    day = sub_soup.select_one('.info_list').find_all('li')[-4].find('span').string
    page = sub_soup.select_one('.info_list').find_all('li')[-3].find('span').string
    price = sub_soup.select_one('.pbr').find('del').string
    cost = sub_soup.select('.pbr')[1].find('strong').string
    mile = sub_soup.select('.pbr')[-1].string
    mile = int(mile.split('점')[0].replace(',',''))

    lines3.append([name, artist, translate, day,convert_units(page), convert_units(price), convert_units(cost), mile])

In [255]:
df3 = pd.DataFrame(lines3, columns =['책제목', '저자', '역자', '출간일', '쪽수','정가','판매가', '마일리지'])
df3.head()

,책제목,저자,역자,출간일,쪽수,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",2021-12-14,416,19800,17820,990
1,"다시, 배우다",폴 김,없음,2021-12-10,304,16000,14400,800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",2021-12-10,376,22000,19800,1100
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",없음,2021-12-01,384,18000,16200,900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",2021-11-30,600,36000,36000,1080


In [256]:
df3.tail()

,책제목,저자,역자,출간일,쪽수,정가,판매가,마일리지
15,나도 영문 손글씨 잘 쓰면 소원이 없겠네,윤정희(리제 캘리그라피),없음,2021-11-05,240,16800,15120,840
16,구글 BERT의 정석,수다르산 라비찬디란,"전희원 , 정승환 , 김형준",2021-11-03,368,34000,30600,1700
17,1000개 숨은그림찾기 몬스터 : 찾아도 찾아도 끝판왕,김확실,없음,2021-11-02,96,8000,7200,400
18,1000개 숨은그림찾기 보물찾기 : 찾아도 찾아도 끝판왕,전진희,없음,2021-11-02,96,8000,7200,400
19,1000개 숨은그림찾기 탈것 : 찾아도 찾아도 끝판왕,이한이,없음,2021-11-02,96,8000,7200,400


In [260]:
# 저장하는 법 = df2.to_csv('한빛출판사.csv', index=False) 